In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df=pd.read_csv('dementia_patients_health_data.csv')
df_dementia = df.loc[:, [
    'Dementia','Age', 'APOE_ε4', 'Cognitive_Test_Scores',
    'Depression_Status', 'Physical_Activity', 'Family_History'
]]
df_dementia.describe
# 치매 환자들만 뽑음.
df_dementia.head()

,Dementia,Age,APOE_ε4,Cognitive_Test_Scores,Depression_Status,Physical_Activity,Family_History
0,0,60,Negative,10,No,Sedentary,No
1,1,61,Positive,1,No,Moderate Activity,No
2,0,69,Negative,8,No,Moderate Activity,Yes
3,1,78,Negative,5,Yes,Mild Activity,Yes
4,1,77,Positive,0,No,Mild Activity,Yes


In [44]:
print(df_dementia.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Dementia               1000 non-null   int64 
 1   Age                    1000 non-null   int64 
 2   APOE_ε4                1000 non-null   object
 3   Cognitive_Test_Scores  1000 non-null   int64 
 4   Depression_Status      1000 non-null   object
 5   Physical_Activity      1000 non-null   object
 6   Family_History         1000 non-null   object
dtypes: int64(3), object(4)
memory usage: 54.8+ KB
None


In [46]:
df_Act=df_dementia.groupby(['Depression_Status','Dementia','Physical_Activity']).size()
print(df_Act)

Depression_Status  Dementia  Physical_Activity
No                 0         Mild Activity        182
                             Moderate Activity    160
                             Sedentary            173
                   1         Mild Activity         86
                             Moderate Activity     77
                             Sedentary             77
Yes                1         Mild Activity         83
                             Moderate Activity     81
                             Sedentary             81
dtype: int64


In [30]:
df_rate = df_Act.groupby(level=[0,2]).apply(
    lambda x: x / x.sum()
)
df_rate

Depression_Status  Physical_Activity  Depression_Status  Dementia  Physical_Activity
No                 Mild Activity      No                 0         Mild Activity        0.679104
                                                         1         Mild Activity        0.320896
                   Moderate Activity  No                 0         Moderate Activity    0.675105
                                                         1         Moderate Activity    0.324895
                   Sedentary          No                 0         Sedentary            0.692000
                                                         1         Sedentary            0.308000
Yes                Mild Activity      Yes                1         Mild Activity        1.000000
                   Moderate Activity  Yes                1         Moderate Activity    1.000000
                   Sedentary          Yes                1         Sedentary            1.000000
dtype: float64

In [31]:
df_table = df_Act.unstack(level='Dementia', fill_value=0)
df_table

Dementia                               0   1
Depression_Status Physical_Activity         
No                Mild Activity      182  86
                  Moderate Activity  160  77
                  Sedentary          173  77
Yes               Mild Activity        0  83
                  Moderate Activity    0  81
                  Sedentary            0  81

In [ ]:
from scipy.stats import chi2_contingency
import numpy as np

def cramers_v(x, y):
    table = pd.crosstab(x, y)
    chi2, p, dof, expected = chi2_contingency(table)
    n = table.sum().sum()
    return np.sqrt(chi2 / (n * (min(table.shape) - 1)))

In [ ]:


# 신체활동과 치매의 연관성 카이제곱
table_pa = pd.crosstab(df_dementia['Physical_Activity'], df_dementia['Dementia'])
chi2, p, dof, ex = chi2_contingency(table_pa)

print(table_pa)
print("Chi-square:", chi2)
print("p-value:", p)

Dementia             0    1
Physical_Activity          
Mild Activity      182  169
Moderate Activity  160  158
Sedentary          173  158
Chi-square: 0.2740650645488776
p-value: 0.8719418594188298


In [43]:
import numpy as np

expected = ex
observed = table_pa.values

residuals = (observed - expected) / np.sqrt(expected)
residuals

array([[ 0.09185648, -0.09465478],
       [-0.29459419,  0.30356865],
       [ 0.19416026, -0.20007513]])

In [ ]:
#우울증과 치매의 연관성
table_dep = pd.crosstab(df_dementia['Depression_Status'], df_dementia['Dementia'])
print(table_dep)

chi2, p, dof, ex = chi2_contingency(table_dep)
print("Chi-square:", chi2)
print("p-value:", p)


Dementia             0    1
Depression_Status          
No                 515  240
Yes                  0  245
Chi-square: 341.8501573703128
p-value: 2.5231461865260466e-76


In [39]:
import numpy as np

expected = ex
observed = table_dep.values

residuals = (observed - expected) / np.sqrt(expected)
residuals


array([[  6.39876592,  -6.59369673],
       [-11.23276458,  11.57495742]])